In [1]:
import os
import sys
import scipy
import datetime

In [2]:
sys.path.append(os.path.join(os.pardir, 'src'))

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [4]:
mnist = tf.keras.datasets.mnist

In [5]:
from tqdm.notebook import tqdm
from collections import defaultdict

In [6]:
from sklearn.model_selection import train_test_split
from CBO.distributions import Normal
from CBO.functions import rastrigin, rastrigin_c, square
from CBO.minimize import minimize
from CBO.visualizations import visualize_trajectory_convergence
from CBO.train import train, NeuralNetworkObjectiveFunction, TensorboardLogging, compute_model_dimensionality

In [7]:
def prepare_data():
    X = np.linspace(-100, 100, 1000)
    y = ((np.sign(X) + 2) / 2).astype(int)
    return X.reshape(-1, 1), y

In [8]:
X, y = prepare_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True)

In [9]:
def build_model():
    return tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(3),
    ])

In [10]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
model = build_model()

In [12]:
def conduct_tf_training(model, X_train, X_test, y_train, y_test, epoches=10):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    
    model.fit(x=X_train, y=y_train, epochs=epoches, validation_data=(X_test, y_test))
    
    return model

In [13]:
conduct_tf_training(model, X_train, X_test, y_train, y_test)

Epoch 1/10
16/16 [==============================] - 1s 20ms/step - loss: 6.8483 - sparse_categorical_accuracy: 0.5140 - val_loss: 6.3523 - val_sparse_categorical_accuracy: 0.4860
Epoch 2/10
16/16 [==============================] - 0s 3ms/step - loss: 4.8707 - sparse_categorical_accuracy: 0.5180 - val_loss: 4.1914 - val_sparse_categorical_accuracy: 0.4920
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 2.9915 - sparse_categorical_accuracy: 0.5200 - val_loss: 2.2456 - val_sparse_categorical_accuracy: 0.5020
Epoch 4/10
16/16 [==============================] - 0s 3ms/step - loss: 1.3229 - sparse_categorical_accuracy: 0.5360 - val_loss: 0.5857 - val_sparse_categorical_accuracy: 0.5560
Epoch 5/10
16/16 [==============================] - 0s 5ms/step - loss: 0.2657 - sparse_categorical_accuracy: 0.9100 - val_loss: 0.1121 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0938 - sparse_categorical_accuracy:

In [16]:
cbo_model, trajectory = train(model, loss, X_train, y_train, n_particles=200, 
                              time_horizon=2000,
                              optimizer_config = {'alpha': 50, 'sigma': 0.4**0.5, 'dt': 0.1}, 
                              initial_distribution=Normal(0, 1),
                              return_trajectory=True, verbose=True, particles_batches=10, 
                              dataset_batches=10, X_val=X_test, y_val=y_test, 
                              tensorboard_logging=None, cooling=True,
                              update_all_particles=True)

KeyboardInterrupt: 